In [1]:
# TODO: Change once API regarding exports stabilizes
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().parent.parent))

import numpy as np
import sympy as sp
from src.systems.base.core.continuous_symbolic_system import ContinuousSymbolicSystem

In [2]:
class SymbolicPendulum(ContinuousSymbolicSystem):
    def define_system(
        self,
        m_val: float = 1.0,
        l_val: float = 1.0,
        beta_val: float = 1.0,
        g_val: float = 9.81,
    ):
        """First order model of a pendulum"""
        # define the symbolic variables
        theta, theta_dot = sp.symbols("theta theta_dot", real=True)
        u = sp.symbols("u", real=True)
        m, l, beta, g = sp.symbols("m l beta g", real=True, positive=True)

        # add the variables to the system fields properly
        self.parameters = {m: m_val, l: l_val, beta: beta_val, g: g_val}
        self.state_vars = [theta, theta_dot]
        self.control_vars = [u]
        self.order = 1

        # define the dynamics of the system
        ml2 = m * l * l
        self._f_sym = sp.Matrix(
            [theta_dot, (-beta / ml2) * theta_dot + (g / l) * sp.sin(theta) + u / ml2],
        )
        self._h_sym = sp.Matrix([theta])

    def setup_equilibria(self):
        # method to add equilibria to the system automatically after initialization

        # add the stable equilibrium where the pendulum is hanging down
        self.add_equilibrium(
            "downward",
            x_eq=np.array([0.0, 0.0]),
            u_eq=np.array([0.0]),
            verify=True,
            )

        # add the unstable equilibrium where the pendulum is inverted
        self.add_equilibrium(
            "inverted",
            x_eq=np.array([np.pi, 0.0]),
            u_eq=np.array([0.0]),
            stability="unstable",
            notes="Requires active control",
            )

# Instantiate the system
pendulum = SymbolicPendulum()

# Initial conditions
x0 = np.array([1.0, 0.0])

In [3]:
# Integrate a trajectory with auto-selection of
# relevant arguments
pendulum_integration_result = pendulum.integrate(
        x0=x0,
        t_span = (0.0, 15.0),
    )
print(pendulum_integration_result["x"])

[[ 1.00000000e+00  0.00000000e+00]
 [ 1.00000594e+00  9.89506734e-03]
 [ 1.00071537e+00  1.08212210e-01]
 [ 1.01616061e+00  5.07153694e-01]
 [ 1.06848383e+00  1.02955621e+00]
 [ 1.17671541e+00  1.63677770e+00]
 [ 1.36135156e+00  2.32255220e+00]
 [ 1.64282360e+00  3.05918535e+00]
 [ 2.03894953e+00  3.75224346e+00]
 [ 2.39769093e+00  4.10243396e+00]
 [ 2.77307091e+00  4.19097647e+00]
 [ 3.12306614e+00  3.99941533e+00]
 [ 3.44946098e+00  3.54918825e+00]
 [ 3.74593616e+00  2.85135623e+00]
 [ 3.98806126e+00  1.95098168e+00]
 [ 4.13669483e+00  1.01563199e+00]
 [ 4.19214004e+00  1.63426037e-01]
 [ 4.17347014e+00 -5.71812199e-01]
 [ 4.09367426e+00 -1.21935449e+00]
 [ 3.89225819e+00 -1.94846312e+00]
 [ 3.66106240e+00 -2.34050770e+00]
 [ 3.40784356e+00 -2.47861435e+00]
 [ 3.17389337e+00 -2.38305962e+00]
 [ 2.96897566e+00 -2.11414159e+00]
 [ 2.79926395e+00 -1.72230825e+00]
 [ 2.67127049e+00 -1.26032604e+00]
 [ 2.58714906e+00 -7.77560678e-01]
 [ 2.54434775e+00 -3.12898845e-01]
 [ 2.53608001e+00  5

In [4]:
STATE_NAMES = ["angle", "angular velocity"]
trajectory_plot = pendulum.plot(
    result=pendulum_integration_result,
    state_names = STATE_NAMES,
)
trajectory_plot.show()

In [ ]:
upright_point = pendulum.get_equilibrium("inverted")
downward_point = pendulum.get_equilibrium("downward")
print(upright_point)
print(downward_point)
phase_portrait = pendulum.phase_plotter.plot_2d(
    x=pendulum_integration_result["x"],
    state_names = STATE_NAMES,
    show_direction = True,
    equilibria=[upright_point,
                downward_point],
)
phase_portrait.show()
# TODO: fix issue with equilibria not being marked on the plot

(array([3.14159265, 0.        ]), array([0.]))
(array([0., 0.]), array([0.]))
